<a href="https://colab.research.google.com/github/comp0161/colab/blob/main/COMP0161_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Music with Deep Learning (Part 1)

In this and the next two lab sessions we will train a **deep neural network** to generate something resembling baroque music, synthesize the results and finally use some audio effects to change how it sounds.

All three sessions will make use of [Google's Colab computing environment](https://colab.research.google.com/#), using [Python](https://docs.python.org/3/tutorial/index.html) code for most of the processing. If you have not used Colab before you can read more about it [here](https://colab.research.google.com/notebooks/basic_features_overview.ipynb). A very brief summary of what you need to know can be found in the section below.

You do not need to know Python to complete the labs, but if you are comfortable with coding and wish to explore further there a few additional tasks suggested that you might like to try.

## Using Colab

Colab provides free access to Google computational resources via a web-based front end known as a **notebook**. The notebook content is broken down into **cells**, of which there are two basic types: 
* **Text** cells (like this one) contain rich text and possibly other media such as images. You should **read** the content of these cells!
* **Code** cells (like the one below) contain program instructions in Python, and sometimes other operating system commands. The latter are typically prefixed by an exclamation mark, like this: `!ls -l`
   
   Code content can be executed on the remote **virtual machine** by clicking on the **play**  ( ▶︎ ) button that appears in the top left hand corner of the cell when you move your cursor over it. Some code may take a while to run — the play button will change to a **stop** (◾️) button, and an animated progress wheel will show around it. As the code runs, it may produce output, which will appear at the bottom of the cell. You can click the stop button to halt the execution.

  (The notebook must be **connected** to a virtual machine in order to run code cells. This should happen automatically the first time you attempt to execute a cell — there will be a delay while this takes place.) 

Try clicking the play button on the cell below to run it.

In [ ]:
# a very simple example of a code cell
# running the cell will execute the statement below and generate output
print('hello world!')

There are two important caveats to be aware of, one to do with the Jupyter notebook interface and the other to do with Colab:

1. The notebook interface allows **out of order execution** of code cells. That is, it is possible to run later cells before earlier ones. This is really **never what you want** and can give rise to all kinds of problems with inconsistent state. Always be sure to run cells in order. If you need to go back and re-run earlier cells (eg, because you want to change something) always follow that up by re-running all the subsequent cells in order too, to make sure everything is consistent.

2. The virtual machine environment that Colab runs behind this notebook interface is **resource-limited** and **transient**. If you don't do anything for awhile, the notebook may disconnect from the virtual machine. If you use too much computation then your session may be halted — and what qualifies as "too much" is kept intentionally vague. VMs are purged daily: if you stop and come back to it tomorrow the VM will have been deleted and you will need to go back to the beginning and go through the setup steps again. If you generate something that you want to keep, you should download to your local machine.


# Background

Although the overall goal of these labs is to use deep learning to generate music, in this first session we will not deal with the neural network model itself, and instead focus on acquiring, understanding and preparing the data that the network will need in order to learn.

## Representing music as data

Music, and audio in general, is an inherently **temporal** medium — it unfolds as a sequence of events over time. Capturing this as data requires finding a representation for such event sequences.

There are two popular strategies that can be employed for this. A **physical** representation attempts to capture the actual vibrational content of the sound, while a **symbolic** representation describes (some aspects of) the processes used to produce the sound.

Classic analogue recording methods, like the groove on a vinyl LP, and digitally sampled equivalents, like a CD or MP3, use some form of physical representation. Physical audio is very temporally dense, involving vibrational displacements on the order of fractions of a millisecond. To capture digitally the range of frequencies to which human hearing is sensitive requires sampling at a rate of tens of thousands of Hz. The corresponding data tends to be very large even for short snippets of sound.

Conversely, sheet music notation or a pianola roll are symbolic, providing a set of instructions that may be interpreted by a performer or device to reproduce (some) essential features of the music. Symbolic representation is much more efficient, able to encode complex pieces in a very concise way, but it is also very **lossy**. Many details of the sound are simply left unspecified. Notably, different performers might play the same musical score in very different ways.

For our current purposes, we are interested in learning (some of) the bare elements of a piece of music, rather than the sonic nuances of any particular recording, so we will consider only a symbolic representation. We will come back to the processing of physical audio data in Lab 3.

There are numerous ways to represent music symbolically on a computer, but one popular method is [MIDI](https://en.wikipedia.org/wiki/MIDI), the *musical instrument digital interface* standard. Originally designed to enable interoperation of electronic music equipment from disparate manufacturers, MIDI data consists of streams of messages that instruct the receiving devices to do things (such as starting and stopping playing musical notes, but also changing equipment settings) at specific times. As well as specifying how the inter-device communications work, the MIDI standard defines a format for encoding all the messages that make up a particular piece of music into a single `.mid` file.

As with different performers of a musical score, different devices may respond to MIDI messages differently, so a MIDI file does not uniquely define an audio output. There is a loose standard known as General MIDI that defines a common set of instruments, but the actual sounds produced by different implementations can vary widely.

MIDI is not the only (or necessarily the best) interchange format for music data, but it is extremely widely used and supported, and there are a lot of MIDI files out there — including the classical music dataset we're going to use here.

## The [Classical Music MIDI](https://www.kaggle.com/datasets/soumikrakshit/classical-music-midi) dataset

Machine learning is extremely dependent on data for training and evaluation. There are many potential sources of data on the internet, but in this case we will make us of [Kaggle](https://kaggle.com/), a popular hub of datasets, code and discussion, much of it appearing in the context of **competitions**, where individuals and groups vie to solve data analysis problems.

Among the data available on Kaggle is [Classical Music MIDI](https://www.kaggle.com/datasets/soumikrakshit/classical-music-midi), which as the name suggests is a repository of classical music — specifically solo piano pieces — by 18 different composers. The dataset is freely downloadable, but you will need to create a Kaggle account if you don't have one already.

# Setting up

## Setup music handling

In order to load and manipulate MIDI data we'll use the [Music21](https://web.mit.edu/music21/) library, a full-featured music processing and analysis package capable of much more than the fairly trivial uses we'll make of it here.

Music21 supports the generation of music notation via an outside helper application, which for our purposes will be [MuseScore](https://musescore.org/en). It also supports writing music file formats — including MIDI — and also (newly) playback of MIDI in Colab. As a result we don't strictly need a separate audio synthesis engine (yet), but we'll install [FluidSynth](https://www.fluidsynth.org) for completism.

**Note**: installing these packages via `apt-get` generates *a lot* of mostly uninteresting output. To avoid clogging up your browser with all that, the commands below redirect the output to the "throw away" destination `/dev/null`. Bur if you encounter problems with the installation, you may want to remove the `> /dev/null` and `&> /dev/null` to help diagnose what's going wrong.

In [ ]:
# software for rendering music notation
print('installing musescore')
!yes | add-apt-repository ppa:mscore-ubuntu/mscore3-stable > /dev/null
!apt update &> /dev/null
!apt install musescore3 &> /dev/null
print("done")

# software for rendering MIDI to WAV
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
print('done')

# install the music21 package for reading and transforming the MIDI training data
# (an older version seems to be already installed on Colab as of this writing, but
# we want to be up to date)
print('upgrading music21')
%pip install --upgrade music21 > /dev/null
print('done')

## Getting the data

The [Classical Music MIDI](https://www.kaggle.com/datasets/soumikrakshit/classical-music-midi) dataset from Kaggle is free to download, but you'll need to provide an **account key** for the Kaggle API. If you do not already have a Kaggle account, you can create one [here](https://www.kaggle.com). Once you have done so, download the account key from your user account -- see instructions [here](https://github.com/Kaggle/kaggle-api#api-credentials). You should wind up with a file called `kaggle.json`.

When you run the cell below you should see an option to **choose files** — use this to upload your `kaggle.json` file. This will then be used to download and unzip the dataset. You should wind up with a directory called `classical_music_midi` containing all the MIDI files arranged into subdirectories by composer.

In [ ]:
# setup kaggle API for data downloading
%pip install -q kaggle

from google.colab import files

# upload your API key file when prompted
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# download and unzip the classical music dataset
!kaggle datasets download -d soumikrakshit/classical-music-midi
!mkdir classical_music_midi
!unzip classical-music-midi.zip -d classical_music_midi

# check the directory content
!ls classical_music_midi

## Python library imports

Executing this preamble makes various useful libraries, including Music21, available for use in our later Python code.

In [ ]:
# standard imports that are very widely used
import sys, os, os.path
import copy
import numpy as np
import numpy.random
import json

# more specialised imports for this particular task
import music21 as MU
from IPython.display import Image, Audio

## Configuration

The variables below specify details of which data to use and what preprocessing to perform on it. You should leave these at their defaults to begin with, but are welcome to try out other values if you wish.

In [ ]:
# configuration variables

# configure a random number generator
# changing the value of SEED will produce a different sequence
# but NB in the default configuration randomisation is not used
SEED = 9907
shared_rng = numpy.random.default_rng(seed=SEED)

# dataset configuration
# you should leave these two settings unchanged unless you
# are explicitly importing your own data to use instead
MIDI_DATA = 'classical_music_midi'
COMPILED_DATA = 'data'

# comma-separated list of composers to use when building the training set
# names should match subfolders in the MIDI_DATA directory
# note that the amount of data available varies a lot between composers,
# some names (eg, `beeth`) will significantly increase processing time
# and produce a much larger data file
COMPOSERS = 'bach,mozart,haydn'

# pre-processing settings (see relevant section below for more information)
# maximum number of notes in a chord
SIMPLIFY_LIMIT = 3
# how to drop notes when simplifying chords?
SIMPLIFY_MODE = 'low'
# remove tied notes from subsequent chords?
STRIP_TIES = True

# how many chords tokens per line in the output file
# (this is largely cosmetic but will affect readability)
TOKENS_PER_LINE = 8

# filenames for intermediate data
MUSIC_MID = 'music.mid'
MUSIC_WAV = 'music.wav'

## Display helpers

Jupyter notebook (the software that produces the web page you're looking at now) doesn't know how to show or play music, but Music21 does, so we'll mostly use that. We'll also provide a converter for rendering audio with FluidSynth. We'll mainly use this in lab 3, though.

In [ ]:
# helper function for playing music in the notebook
# note that this is probably only useful for fairly small pieces


def fluid_play(music, rate=22050, midi_name=MUSIC_MID, wav_name=MUSIC_WAV):
  """
  Write music to MIDI, then render that to WAV and display inline as Audio.

  Note: if `None` is passed to `midi_name`, Music21 will invent a temp name.
  """
  filename = music.write('mid')
  os.rename(filename, midi_name)
  !fluidsynth -ni font.sf2 $midi_name -F $wav_name -r $rate > /dev/null
  display(Audio(wav_name))

Run the following cell to check that the display functions work. Note that the Music21 `show('midi')` playback sounds quite different to the `fluid_play` version (and louder).

In [ ]:
snippet = MU.converter.parse("tinynotation: 3/4 c4 d8 f g16 a g f#")

# show using music21 built-in capabilities
snippet.show('text')
snippet.show()
snippet.show('midi')

# synthesise and play using fluidsynth
fluid_play(snippet)

# Data processing


## Musical language modelling

We're going to be treating our learned music generation task as one of [language modelling](https://en.wikipedia.org/wiki/Language_model): given a large body of token sequences, learn a probability distribution over the next token in a sequence. To do so, we need to convert our MIDI dataset into sequences of tokens -- for simplicity we will represent these as **text** tokens, which we can think of as **words**.

## Chords as words

Music sequences (in the case of MIDI and also in general) can contain a variety of units of information (you can see some examples in the output of the cell above), but we will limit ourselves to just **chords** and **rests**, each with an associated **duration**. (For simplicity we'll consider single notes also to be chords.) A "word" in our musical language will be a set of zero or more simultaneous notes plus how long that set is played.

Note pitches are represented in a pretty standard textual way by Music21, and we'll just use those as they come. We'll special case the empty set with the symbol `rest`. Durations are represented numerically, as a fraction of a **quarter note** (or crochet). Adding in some punctuation to delimit the elements within each token, we wind up with words that look like this:

```G3.C4.E4;0.25 rest;0.25 G1.B3.D4;0.25 G1.B3.D4;0.25```

The functions `music_to_text` and `text_to_music` below implement conversions between this tokenised text representation and the `Stream` structure used by Music21. Note that because we're only retaining notes and rests, a lot of stuff gets lost in translation here.

## Enforcing serialisation

Implicit in the above is that our words are purely sequential -- there is no scope for multiple words occurring in parallel. All simultaneous activity will be marshalled into a single chord.

Music21 has a handy function `chordify` for converting music streams into serial event sequences, so we'll use that to produce our token sequences. However, Music21's chordified sequences still allow for the possibility of overlapping notes of different durations through the mechanism of [ties](https://hellomusictheory.com/learn/tied-notes/) — a notational convention for joining consecutive notes of the same pitch into a single continous longer one.

If some notes in a chord are tied and others aren't then we wind up with confusing overlaps. We are going to be more restrictive, because we don't want to have to think about notes extending beyond word boundaries. We'll require all notes in a chord to start at the same time and have the same duration. Again, Music21 provides a useful function, `stripTies`, for collapsing tied notes in sequences, but unfortunately it doesn't work how we want for individually tied notes within chords — it leaves a few unwanted notes around in some cases. So we define our own `strip_ties` function below to do the extra pruning. This will sacrifice some of the information in the music in favour of simplicity, but for our purposes it's mostly not *too* damaging.

## Vocabulary reduction

When dealing with language modelling problems, we use the concept of a **vocabulary**, which refers to the collection of all the possible unique words used in our dataset. In any language modelling problem, we need to make sure that that our total vocabulary is well covered by our training set. For example, if we have a very large vocabulary but a small training set, there might be some words that never appear in the training set at all, or appear very rarely. When these words turn up in the test set (or in a real world application of our model) our model won’t be good at handling those words because it won't have had much experience of them.

In our case here, each word in our vocabulary is a different note combination. We have potentially a huge number of different note combinations, many of which may be used only rarely, leading to a language with a large vocabulary that is only weakly covered by our training set. One way we can reduce this, at least *a little bit*, is to limit the number of notes allowed in any chord. Again, this involves throwing out potentially important information, but it does make our task a bit more tractable given a limited amount of data. The `simplify` function below implements this approach.

## Putting it together

Finally, we bundle all this functionality together into the functions `tokenise`, which converts a single MIDI file into text token form, and `build_dataset`, which compiles a corpus of tokenised music from the Classical Music MIDI dataset for some chosen subset of composers (or all of them).

In [ ]:
# functions for music processing

def strip_ties ( s, inPlace=True ):
    """
    Strip tied chords and drop non-starting tied notes from within chords.
    NB: operates in place by default.

    Intended for chordified streams, will probably produce weird
    results otherwise.
    """
    if not inPlace:
        s = copy.deepcopy(s)
    
    s.stripTies(inPlace=True)
    
    for element in s.flat:
        if isinstance(element, MU.chord.Chord):
            deletions = []
            for note in element:
                if note.tie is not None:
                    if note.tie.type == 'start': note.tie = None
                    else: deletions.append(note.pitch.nameWithOctave)
            
            for note in deletions:
                element.remove(note)
    
    return s


def simplify ( s, limit=SIMPLIFY_LIMIT, mode=SIMPLIFY_MODE, rng=shared_rng, inPlace=True ):
    """
    Drop notes from big chords so they have no more than `limit` notes.
    NB: operates in place by default.

    Intended for chordified streams, will probably produce weird
    results otherwise.
    
    Drop strategies are pretty dumb. We always keep the highest and lowest notes
    (crudely assumed to be melody and bass respectively). Notes are dropped from
    the remainder according to one of three strategies:
    
        'low': notes are dropped from low to high (the default)
        'high': notes are dropped from high to low
        'random': notes are dropped randomly
    
    Latter could actually increase vocab by mapping the same input chord
    to several outputs. Modes can be abbreviated to initial letters.
    """
    if limit < 2: limit = 2

    if not inPlace:
        s = copy.deepcopy(s)
    
    drop_func = {
                    'r' : lambda d, c: rng.choice(d, c, replace=False),
                    'h' : lambda d, c: d[(len(d)-c):]
                }.get(mode.lower()[0],
                      lambda d, c: d[:(c-len(d))])
    
    for element in s.flat:
        if isinstance(element, MU.chord.Chord):
            if len(element) > limit:
                drop_count = len(element) - limit
                drops = [ nn.pitch.nameWithOctave for nn in element ][1:-1]
                
                if len(drops) > drop_count:
                    drops = drop_func(drops, drop_count)
                
                for note in drops:
                    element.remove(note)
    
    return s


def music_to_text ( s ):
    """
    Convert music stream into a list of text tokens defining the
    chords, notes and rests and their durations.
    
    Intended for chordified streams, will probably produce weird
    results otherwise.
    """
    result = []
    for element in s.flat:
        name = None
        if isinstance(element, MU.chord.Chord):
            name = '.'.join(n.nameWithOctave for n in element.pitches)
        elif isinstance(element, MU.note.Rest):
            name = 'rest'
        elif isinstance(element, MU.note.Note):
            name = str(element.nameWithOctave)
    
        if name is not None:
            # convert any stray empty notes or chords into rests
            name = name or 'rest'
            result.append(f'{name};{float(element.duration.quarterLength):.6g}')

    return result


def text_to_music( t ):
    """
    Convert a sequence of text tokens into a music stream.
    """
    result = MU.stream.Stream()

    for element in t:
        notes, quarters = element.split(';')
        duration = MU.duration.Duration(float(quarters))
    
        if '.' in notes:
            notes = notes.split('.')
            chord = []
            for nn in notes:
                note = MU.note.Note(nn)
                note.duration = duration
                chord.append(note)
            result.append(MU.chord.Chord(chord))
        elif notes == 'rest':
            note = MU.note.Rest()
            note.duration = duration
            result.append(note)
        else:
            note = MU.note.Note(notes)
            note.duration = duration
            result.append(note)

    return result

def tokenise ( file, simplify_limit=SIMPLIFY_LIMIT, simplify_mode=SIMPLIFY_MODE,
               do_strip=STRIP_TIES, rng=shared_rng ):
    """
    Read a MIDI file and convert to text tokens, with
    optional preprocessing.
    """
    raw_stream = MU.converter.parse(file)
    chorded = raw_stream.chordify()
    
    if do_strip:
        strip_ties(chorded)
    
    if simplify_limit:
        simplify(chorded, simplify_limit, simplify_mode, rng=rng)
    
    return music_to_text(chorded)


def build_dataset ( midi_path=MIDI_DATA, composers=COMPOSERS, out_path=None,
                    do_strip=STRIP_TIES, simplify_limit=SIMPLIFY_LIMIT,
                    simplify_mode=SIMPLIFY_MODE, rng=shared_rng ):
    """
    Construct a tokenised data file of optionally simplified and tie-stripped
    music by the specified composers, for use in training a language model.
    """
    if composers:
        comps = composers.split(',')
    else:
        comps = [ ff for ff in os.listdir(midi_path) if os.path.isdir(os.path.join(midi_path, ff)) ]
    
    if out_path is None:
        out_path = os.path.join(COMPILED_DATA, '_'.join(comps) + '.txt')
    
    out_dir = os.path.split(out_path)[0]
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    
    tokens = []
    count = 0

    for comp in comps:
        dir = os.path.join(midi_path, comp)
        for filename in os.listdir(dir):
            if filename.lower().endswith('.mid'):
                print(f'reading {filename}')
                tokens.extend(tokenise(os.path.join(dir, filename),
                                       simplify_limit=simplify_limit,
                                       simplify_mode=simplify_mode,
                                       do_strip=do_strip,
                                       rng=rng))
                count += 1
    
    print(f'loaded {len(tokens)} tokens from {count} files')

    print(f'writing tokens to {out_path}')

    with open(out_path, 'w') as f:
        for off in range(0, len(tokens), TOKENS_PER_LINE):
            print(' '.join(tokens[off:off+TOKENS_PER_LINE]), file=f)
    

    
    

## Check it works


In [ ]:
# reading and displaying
bach = MU.converter.parse('classical_music_midi/bach/bach_846.mid')
bach_snip = MU.stream.Stream(bach.flatten()[:100])
bach_snip.show()
bach_snip.show('midi')

In [ ]:
# stripping & simplifying
bach_s = strip_ties(bach.chordify(), inPlace=False)
bach_s = simplify(bach_s)
bss = MU.stream.Stream(bach_s.flatten()[:100])
bss.show()
bss.show('midi')

In [ ]:
# mapping to and from text
# (note how information such as tempo gets lost in translation)
bssr = text_to_music(music_to_text(bss))
bssr.show()
bssr.show('midi')

# Build a dataset

With all that in place, let's actually build a dataset that we'll be able to use to learn from.

(Note that this process is **slow**. In my tests building the `bach,mozart,haydn` dataset took about 10 minutes. Building a dataset for all the composers might not be practical on Colab -- it will at least take a *long* time.)

In [ ]:
build_dataset()

Check that the output looks sane. (Note that if you have changed the build configuration, you will need to edit the below command to match your built file.)

In [ ]:
!head data/bach_mozart_haydn.txt

## Download

Finally, download the dataset file for use in Lab 2. Again, edit this command to match your actual dataset if you've changed the configuration.

(You can also download using the file browser, accessed via the folder icon in the sidebar.)

In [ ]:
files.download('data/bach_mozart_haydn.txt')

# Discussion

While you are waiting for your dataset to compile, here are some things to think & talk about:

* How reasonable is it to think of music in the sort of language terms described above?
* In defining our translation from MIDI to a word-like text encoding we have had to make a number of choices: what to include, what to leave out. What do you think of these choices? Are they reasonable or are there obvious missteps?
* How might these choices be expected to affect or **bias** what our machine learning algorithm is able to discover and model?

# Further work

If you have time, interest and/or aptitude to spare, here are some additional tasks you might like to try:

* Modify the configuration to use a different set of composers, change the chord sizes, pruning method etc.
* Locate and use some other source of MIDI data rather than this set of classical piano pieces.
* Modify the music encoding and decoding functions to include additional info, such as the tempo or key. (NB: if you do this you will need to hang onto your modified code to use in Lab 2.)